# Testing/debugging PQMF

## Warning: outdated

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import tensorflow as tf

from thesis import pqmf
from ddsp.colab.colab_utils import (
    auto_tune, get_tuning_factor, download,
    play, record, specplot, upload, audio_bytes_to_np,
    DEFAULT_SAMPLE_RATE)
import numpy as np
import IPython
import einops
import gin
gin.enter_interactive_mode()

def play_audio(audio):
    audio = np.array(audio)
    audio = np.squeeze(audio)
    IPython.display.display(IPython.display.Audio(audio, rate=16000))

from ddsp.losses import SpectralLoss

def get_loss(target_audio, audio):
    loss_class = SpectralLoss()
    loss = loss_class(tf.convert_to_tensor(target_audio), tf.convert_to_tensor(audio))
    return float(loss)

In [ ]:
sample_rate = 16000
bands = 8
attenuation = 100

input_f = open("../data/audio/violin/II. Double.mp3", "rb")
wav_bytes = input_f.read()
audio = audio_bytes_to_np(wav_bytes)
audio = audio[:sample_rate * 10]
audio_np = audio.copy()

if len(audio.shape) == 1:
    audio = audio[np.newaxis, :, np.newaxis]

audio_tf = tf.convert_to_tensor(audio)
audio_torch = torch.Tensor(audio_np).reshape((1, 1, -1))

In [ ]:
play_audio(audio[0,:,0])

In [ ]:
p = pqmf.PQMFBank(attenuation=100, n_bands=4)

In [ ]:
p.hk.shape

In [ ]:
z = p.analysis(audio_tf)

y = p.synthesis(z)

In [ ]:
play_audio(y[0,:,0])

In [ ]:
from thesis import pqmf_rave
pr = pqmf_rave.PQMF(100, 8)

In [ ]:
pr.hk.shape

In [ ]:
def mean_distance(a, b):
    a = np.array(a).squeeze()
    b = np.array(b).squeeze()

    if a.shape[0] < 50:
        a = a.swapaxes(0, 1)

    if b.shape[0] < 50:
        b = b.swapaxes(0, 1)

    abs_distance = np.abs(a - b).mean()
    rel_distance = (np.abs(a - b) / (np.abs(a) + np.abs(b) + 1e-9)).mean()
    print(abs_distance, rel_distance)

In [ ]:
p.hk.shape

In [ ]:
hk_for_torch = torch.Tensor(np.array(einops.rearrange(p.hk, "t 1 m -> m t")))
z_torch_fast = pqmf_rave.polyphase_forward(audio_torch, hk_for_torch)
z_torch_fast = pqmf_rave.reverse_half(z_torch_fast)

In [ ]:
z_torch = pqmf_rave.classic_forward(audio_torch, hk_for_torch)
z_torch = pqmf_rave.reverse_half(z_torch)
z_torch.shape

In [ ]:
# Upsample to hear what it sounds like

from thesis.util import resample

def preview_decomposed(z):
    z2 = resample(z, z.shape[1] * 8)

    for i in range(3):
        play_audio(z2[0,:,i])

In [ ]:
def polyphase_forward(x, hk):
    """
    Polyphase implementation of the analysis process (fast)
    Parameters
    ----------
    x: torch.Tensor
        signal to analyse ( B x T x 1 )

    hk: torch.Tensor
        filter bank ( T x 1 x M )
    """
    x = einops.rearrange(x, "b (t m) c -> b t (c m)", m=hk.shape[-1])
    # if rearrange_filter:
    # hk = einops.rearrange(hk, "c (t m) -> c m t", m=hk.shape[0])

    hk = einops.rearrange(hk, "(t m) 1 c -> t m c", m=hk.shape[-1])
    # print(x.shape)
    # print(hk.shape)
    # x = nn.functional.conv1d(x, hk, padding=hk.shape[-1] // 2)[..., :-1]
    #
    # # shape of x: [batch, in_width, in_channels]
    # # shape of hk: [filter_width, in_channels, out_channels]
    x = tf.nn.conv1d(x, hk, stride=1, padding="SAME")

    # x = pqmf.reverse_half(x)

    return x

zpoly = polyphase_forward(audio_tf, p.hk)
print(zpoly.shape)

mean_distance(zpoly, z)
mean_distance(zpoly, z_torch_fast)
mean_distance(z_torch, z_torch_fast)
mean_distance(z, z_torch)

In [ ]:
def polyphase_inverse(x, hk):
    """
    Polyphase implementation of the synthesis process (fast)
    Parameters
    ----------
    x: torch.Tensor
        signal to synthesize from
        torch: ( B x 1 x T )
        us: B x T x 1

    hk: torch.Tensor
        filter bank
        torch: ( M x T )
        us: T x 1 x M
    """

    m = hk.shape[-1]

    hk = tf.reverse(hk, [0])

    hk = einops.rearrange(hk, "(t m) 1 c -> t c m", m=m)

    #pad = hk.shape[-1] // 2 + 1
    #x = nn.functional.conv1d(x, hk, padding=int(pad))[..., :-1] * m

    x = tf.nn.conv1d(x, hk, stride=1, padding="SAME")

    x = tf.reverse(x, [1])
    x = einops.rearrange(x, "b t (c m) -> b (t m) c", m=m)
    x = tf.reverse(x, [1])

    # x = einops.rearrange(x, "b t (c m) -> b (t m) c", m=m)

    # x = x[..., 2 * hk.shape[1] :]
    return x

ypoly = polyphase_inverse(zpoly, p.hk)
ypoly.shape

In [ ]:
play_audio(ypoly)

In [ ]:
%%timeit -n 100

zpoly = polyphase_forward(audio_tf, p.hk)
ypoly = polyphase_inverse(zpoly, p.hk)

In [ ]:
%%timeit -n 100

zold = p.analysis(audio_tf)
yold = p.synthesis(zold)

In [ ]:
mean_distance(zpoly, zold)

In [ ]:
import os

import einops
import tensorflow as tf
from ddsp.losses import SpectralLoss
from ddsp.colab.colab_utils import audio_bytes_to_np

from thesis.pqmf import PQMFBank


def get_loss(target_audio, audio):
    loss_class = SpectralLoss()
    loss = loss_class(tf.convert_to_tensor(target_audio), tf.convert_to_tensor(audio))
    return float(loss)


def get_audio_fixture():
    input_f = open(
        os.path.join("../fixtures/violin_1s.wav"), "rb"
    )
    wav_bytes = input_f.read()
    audio = audio_bytes_to_np(wav_bytes)
    return audio


audio = get_audio_fixture()
audio = tf.convert_to_tensor(audio)

# Split into two halves to have a batch size of more than 1
batch_size = 2
audio = einops.rearrange(audio, "(b t) -> b t 1", b=batch_size)
n_samples = audio.shape[1]

bands = 4
attenuation = 100

pqmf = PQMFBank(attenuation=attenuation, n_bands=bands)

analyzed = pqmf.analysis(audio)
# self.assertEqual(
#     analyzed.shape.as_list(), [batch_size, n_samples // bands, bands]
# )

synthesized = pqmf.synthesis(analyzed)
# self.assertEqual(synthesized.shape.as_list(), [batch_size, n_samples, 1])

# The reconstructed audio should be close enough to the original.
# The actual loss value should be <=0.1, so the threshold 1.0 is lenient.
print(get_loss(audio, synthesized))


In [ ]:
specplot(audio[1,:,0])

In [ ]:
specplot(synthesized[1,:,0])

In [ ]:
play_audio(audio[1])

In [ ]:
play_audio(synthesized[1])

In [ ]:
preview_decomposed(zpoly)

In [ ]:
preview_decomposed(z)

In [ ]:
preview_decomposed(np.array(einops.rearrange(z_torch_fast, "1 c t -> 1 t c")))

In [ ]:
rec1 = p1.synthesis(temp)
rec1.shape

In [ ]:
play_audio(rec1)
get_loss(audio_tf1, rec1)

## Comparison

In [ ]:
t0 = np.array(temp0).swapaxes(1, 2)
t1 = np.array(temp1)

In [ ]:
print(((t0 - t1) ** 2).mean())
print(((t0 - t1) ** 2).max())

In [ ]:
((np.array(p1.hk).squeeze().T - np.array(p0.hk)) ** 2).sum()

In [ ]:
p0.hk

In [ ]:
rec2 = p1.synthesis(t0)
print(rec2.shape)
play_audio(rec2)
get_loss(audio_tf1, rec2)

In [ ]:
rec3 = p0.inverse(torch.Tensor(t1.swapaxes(1, 2)))
play_audio(rec3)
get_loss(audio_tf1, rec3.swapaxes(1, 2))

In [ ]:
x = np.zeros(30)
x[::5] = 1
x

## Debugging

In [ ]:
x = t0
x1 = tf.nn.conv1d_transpose(
    x,
    p1.updown_filter * p1.n_band,
    strides=p1.n_band,
    output_shape=(
        tf.shape(x)[0],
        tf.shape(x)[1] * p1.n_band,
        p1.n_band,
    ),
)

x1_torch = torch.Tensor(np.array(x1))

In [ ]:
hk = p0.hk.flip(-1)
y = nn.functional.conv1d(
        x1_torch,
        p0.hk.flip(-1).unsqueeze(0),
        padding=hk.shape[-1] // 2,
    )[..., 1:]